In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.applications import MobileNet, DenseNet169, ResNet50, EfficientNetB2, EfficientNetB4, Xception, InceptionV3, VGG19, DenseNet201, ResNet152
from keras.layers import GlobalAveragePooling2D
import pandas as pd
import numpy as np
import os
from shutil import copy
from zipfile import ZipFile
import matplotlib.pyplot as plt
import cv2
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from matplotlib import pyplot as plt
from matplotlib import image
from sklearn.model_selection import KFold

In [ ]:
def create_model():
  # base_model = MobileNet(weights='imagenet', input_shape=(224,224,3), include_top=False)
  # base_model = DenseNet169(weights='imagenet', input_shape=(224,224,3), include_top=False)
  # base_model = EfficientNetB2(weights='imagenet', input_shape=(224,224,3), include_top=False)
  # base_model = EfficientNetB4(weights='imagenet', input_shape=(224,224,3), include_top=False)
  # base_model = ResNet50(weights='imagenet', input_shape=(224,224,3), include_top=False)
  # base_model = Xception(weights='imagenet', input_shape=(224,224,3), include_top=False)
  # base_model = InceptionV3(weights='imagenet', input_shape=(224,224,3), include_top=False)
  # base_model = VGG19(weights='imagenet', input_shape=(224,224,3), include_top=False)
  # base_model = DenseNet201(weights='imagenet', input_shape=(224,224,3), include_top=False)
  base_model = ResNet152(weights='imagenet', input_shape=(224,224,3), include_top=False)

  for layer in base_model.layers:
    layer.trainable = False

  last_layer = base_model.output
  last_layer = BatchNormalization()(last_layer)
  last_layer = GlobalAveragePooling2D()(last_layer)
  last_layer = Dense(256,activation='relu')(last_layer)
  last_layer = Dense(64,activation='relu')(last_layer)
  last_layer = Dense(16,activation='relu')(last_layer)
  last_layer = Dropout(0.4)(last_layer)
  last_layer = Dense(4,activation='sigmoid')(last_layer)
  model = Model(base_model.input, last_layer)

  model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy','Precision','Recall'])

  model.summary()

  return model

In [ ]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/ResNet152/saved-model-14-acc_0.8889-pre_0.6330-rec_0.9583.hdf5')

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_4 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_13[0][0]']               
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_4[0][0]']       
                                )                                                          

In [ ]:
data_path = "/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/"

In [ ]:
train_set = data_path + 'train/'
val_set = data_path + 'valid/'
test_set = data_path + 'test/'
epochs = 30

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale=1. / 255
)
val_datagen= ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale=1. / 255)
test_datagen= ImageDataGenerator(
    rescale=1. / 255
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_set,
    target_size = (224,224),
    batch_size = 8,
    class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory(
    test_set,
    target_size = (224,224),
    batch_size = 8,
    shuffle=True,
    class_mode = 'categorical')
validation_generator = test_datagen.flow_from_directory(
    val_set,
    target_size = (224,224),
    batch_size = 8,
    shuffle=True,
    class_mode = 'categorical')

Found 613 images belonging to 4 classes.
Found 315 images belonging to 4 classes.
Found 72 images belonging to 4 classes.


In [ ]:
a = model.evaluate(test_generator)

40/40 [==============================] - 79s 2s/step - loss: 0.7598 - accuracy: 0.7651 - precision: 0.5738 - recall: 0.8762


In [ ]:
obj = pd.read_pickle(r'/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/ResNet152/metrics.pickle')
obj

{'loss': 0.759780764579773,
 'accuracy': 0.7650793790817261,
 'precision': 0.5738045573234558,
 'recall': 0.8761904835700989}

In [ ]:

import pickle

# some_obj = {'loss':a[0], 'accuracy':a[1], 'precision':a[2], 'recall':a[3], 'cm':cm}
some_obj = {'loss':a[0], 'accuracy':a[1], 'precision':a[2], 'recall':a[3]}
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'MobileNet/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'DenseNet169/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'EfficientNetB2/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'EfficientNetB4/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'ResNet50/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'Xception/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'InceptionV3/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'VGG19/' +'metrics.pickle', 'wb') as f:
# with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'DenseNet201/' +'metrics.pickle', 'wb') as f:
with open('/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/' + 'ResNet152/' +'metrics.pickle', 'wb') as f:
    pickle.dump(some_obj, f)

In [ ]:
models = ['DenseNet169', 'DenseNet201', 'EfficientNetB4', 'InceptionV3', 'MobileNet', 'ResNet50', 'VGG19', 'Xception']

In [ ]:
data = []
for i in models:
  obj = pd.read_pickle(r'/content/drive/MyDrive/CSE572-DM/Final_Project/new_data/models/'+str(i)+'/metrics.pickle')
  data.append([obj['loss'],obj['accuracy'],obj['precision'],obj['recall']])

In [ ]:
data

[[0.547894299030304,
  0.8539682626724243,
  0.45962733030319214,
  0.9396825432777405],
 [0.759780764579773,
  0.7650793790817261,
  0.5738045573234558,
  0.8761904835700989],
 [0.9955337047576904,
  0.5587301850318909,
  0.4578947424888611,
  0.8285714387893677],
 [0.6247036457061768,
  0.7936508059501648,
  0.4446107745170593,
  0.9428571462631226],
 [0.5594971776008606,
  0.8253968358039856,
  0.6512141227722168,
  0.9365079402923584],
 [1.1238778829574585,
  0.5142857432365417,
  0.32642486691474915,
  0.6000000238418579],
 [0.7064489126205444,
  0.723809540271759,
  0.5861244201660156,
  0.7777777910232544],
 [0.8283866047859192,
  0.6698412895202637,
  0.5910224318504333,
  0.7523809671401978]]

In [ ]:
d = pd.DataFrame(data, columns=['loss','accuracy','precision','recall'], index = models)
# d = d.T

In [ ]:
d

,loss,accuracy,precision,recall
DenseNet169,0.547894,0.853968,0.459627,0.939683
DenseNet201,0.759781,0.765079,0.573805,0.876190
EfficientNetB4,0.995534,0.558730,0.457895,0.828571
InceptionV3,0.624704,0.793651,0.444611,0.942857
MobileNet,0.559497,0.825397,0.651214,0.936508
ResNet50,1.123878,0.514286,0.326425,0.600000
VGG19,0.706449,0.723810,0.586124,0.777778
Xception,0.828387,0.669841,0.591022,0.752381
